# Main

In [1]:
import pandas as pd

# Load datasets (specify header=None if your CSVs don't have headers)
X_train = pd.read_csv("/kaggle/input/multifeatvotpip/X_train.csv", header=None)  # Peptide sequences
train_labels = pd.read_csv("/kaggle/input/multifeatvotpip/label_train.csv", header=None)  # Corresponding labels

# Combine features and labels
train_data = pd.concat([X_train, train_labels], axis=1)
train_data.columns = ['peptide_sequence', 'label']

# Load test data
X_test = pd.read_csv("/kaggle/input/multifeatvotpip/X_test.csv", header=None)
test_labels = pd.read_csv("/kaggle/input/multifeatvotpip/label_test.csv", header=None)
test_data = pd.concat([X_test, test_labels], axis=1)
test_data.columns = ['peptide_sequence', 'label']


In [2]:
!pip install modlamp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.7/176.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 3.8 MB/s eta 0:00:00


In [3]:
!pip install fair-esm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from collections import Counter
from itertools import product

import torch
from transformers import AutoTokenizer, AutoModel
from modlamp.descriptors import GlobalDescriptor

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, matthews_corrcoef, roc_auc_score,
    precision_score, f1_score, accuracy_score
)
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import StratifiedShuffleSplit

# ============================================================
# Config
# ============================================================
THRESH               = 0.4         # default decision threshold (also used during tuning MCC eval)
VAL_SIZE             = 0.10        # 10% validation split for tuning
RANDOM_STATE         = 42
COEF_GRID            = [0.0, 1e-6, 5e-6, 1e-5, 2e-5, 5e-5, 1e-4, 2e-4, 5e-4, 1e-3]

np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# Expect these to be defined by caller:
#   train_data -> DataFrame with ['peptide_sequence','label']
#   test_data  -> DataFrame with ['peptide_sequence', ('label' optional)]
# ============================================================
df = train_data.copy()
df_test = test_data.copy()
df["label"] = df["label"].astype(int)
if "label" in df_test.columns:
    df_test["label"] = df_test["label"].astype(int)

X_seq_train = df['peptide_sequence'].tolist()
X_seq_test  = df_test['peptide_sequence'].tolist()
y_train     = df['label'].values.astype(int)

print("Class counts (train):", Counter(y_train))
classes = np.unique(y_train)
class_weights_array = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weight_dict   = {int(c): w for c, w in zip(classes, class_weights_array)}
print("Class weights:", class_weight_dict)

# ============================================================
# 1) ESM embeddings
# ============================================================
def prepare_esm(model_name="facebook/esm2_t6_8M_UR50D"):
    tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()
    return tokenizer, model

def get_esm_embeddings(sequences, tokenizer, model, batch_size=16):
    embeddings = []
    with torch.no_grad():
        for i in range(0, len(sequences), batch_size):
            batch = sequences[i:i+batch_size]
            toks = [" ".join(s.upper()) for s in batch]
            inputs = tokenizer(toks, return_tensors="pt", padding=True, truncation=True).to(device)
            out = model(**inputs)
            cls = out.last_hidden_state[:, 0, :].detach().cpu().numpy()
            embeddings.append(cls)
    return np.vstack(embeddings) if embeddings else np.zeros((0, model.config.hidden_size), dtype=np.float32)

print("Loading ESM model & tokenizer...")
tok, esm_model = prepare_esm("facebook/esm2_t6_8M_UR50D")
print("Extracting ESM embeddings...")
X_esm_train = get_esm_embeddings(X_seq_train, tok, esm_model, batch_size=16)
X_esm_test  = get_esm_embeddings(X_seq_test,  tok, esm_model, batch_size=16)
esm_features = [f"esm_{i}" for i in range(X_esm_train.shape[1])]
print("X_esm_train:", X_esm_train.shape, "| X_esm_test:", X_esm_test.shape)

# ============================================================
# 2) Multi-scale k-mer frequencies
# ============================================================
def get_kmer_freqs(sequences, ks=[2,3,4]):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    all_features = []
    for k in ks:
        vocab = [''.join(p) for p in product(amino_acids, repeat=k)]
        vocab_idx = {kmer: idx for idx, kmer in enumerate(vocab)}
        feat = np.zeros((len(sequences), len(vocab)), dtype=np.float32)
        for i, seq in enumerate(sequences):
            seq = seq.upper()
            kmers = [seq[j:j+k] for j in range(len(seq) - k + 1)
                     if all(ch in amino_acids for ch in seq[j:j+k])]
            counts = Counter(kmers)
            total = float(sum(counts.values()))
            if total > 0:
                for kmer, c in counts.items():
                    feat[i, vocab_idx[kmer]] = c / total
        all_features.append(feat)
    return np.concatenate(all_features, axis=1) if all_features else np.zeros((len(sequences), 0), dtype=np.float32)

def kmer_feature_names(ks=[2,3,4]):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    names = []
    for k in ks:
        kmers = [''.join(p) for p in product(amino_acids, repeat=k)]
        names += [f'kmer_{k}_{i}' for i in range(len(kmers))]
    return names

print("Extracting multi-scale k-mer features...")
X_kmer_train = get_kmer_freqs(X_seq_train, ks=[2,3,4])
X_kmer_test  = get_kmer_freqs(X_seq_test,  ks=[2,3,4])
kmer_features = kmer_feature_names([2,3,4])
print("X_kmer_train:", X_kmer_train.shape, "| X_kmer_test:", X_kmer_test.shape)

# ============================================================
# 3) Physchem (CORE features only)
# ============================================================
hydro_scale = {
    'A': 1.8, 'C': 2.5, 'D': -3.5, 'E': -3.5, 'F': 2.8, 'G': -0.4,
    'H': -3.2, 'I': 4.5, 'K': -3.9, 'L': 3.8, 'M': 1.9, 'N': -3.5,
    'P': -1.6, 'Q': -3.5, 'R': -4.5, 'S': -0.8, 'T': -0.7, 'V': 4.2,
    'W': -0.9, 'Y': -1.3
}
pKa_basic = {'K': 10.5, 'R': 12.5, 'H': 6.0}
pKa_N = 9.69
helix_pref = set("AEHKLMQR")
sheet_pref = set("VIYFWTC")

def hydrophobic_moment(seq, radians_per_res):
    s = seq.upper()
    if not s: return 0.0
    angles = np.arange(len(s)) * radians_per_res
    h = np.array([hydro_scale.get(a, 0.0) for a in s], dtype=float)
    x = np.sum(h * np.cos(angles))
    y = np.sum(h * np.sin(angles))
    mu = np.sqrt(x*x + y*y) / len(s)
    return float(mu)

def positive_charge_at_pH(seq, pH=7.0, include_Nterm=True):
    s = seq.upper()
    chg = 0.0
    for aa, pKa in pKa_basic.items():
        n = s.count(aa)
        chg += n * (1.0 / (1.0 + 10.0**(pH - pKa)))
    if include_Nterm and s:
        chg += 1.0 / (1.0 + 10.0**(pH - pKa_N))
    return float(chg)

def cleavage_density_trypsin(seq):
    s = seq.upper()
    if not s: return 0.0
    L, sites = len(s), 0
    for i in range(L-1):
        if s[i] in "KR" and s[i+1] != 'P': sites += 1
    if s[-1] in "KR": sites += 1
    return sites / L

def cleavage_density_chymo(seq):
    s = seq.upper()
    if not s: return 0.0
    L, sites = len(s), 0
    for i in range(L-1):
        if s[i] in "FYWL" and s[i+1] != 'P': sites += 1
    if s[-1] in "FYWL": sites += 1
    return sites / L

def cleavage_density_elastase(seq):
    s = seq.upper()
    if not s: return 0.0
    L, sites = len(s), 0
    for i in range(L-1):
        if s[i] in "AVIL" and s[i+1] != 'P': sites += 1
    if s[-1] in "AVIL": sites += 1
    return sites / L

def get_physchem_core(sequences, pH=7.0):
    feats = []
    for seq in sequences:
        s = seq.upper(); L = len(s); L_safe = max(L, 1)
        KD_vals = [hydro_scale.get(a, 0.0) for a in s]
        KD_mean = float(np.mean(KD_vals)) if KD_vals else 0.0
        muH_helix = hydrophobic_moment(s, np.deg2rad(100.0))
        muH_sheet = hydrophobic_moment(s, np.deg2rad(180.0))
        pos_charge = positive_charge_at_pH(s, pH=pH, include_Nterm=True)
        pos_charge_density = pos_charge / L_safe
        f_helix = sum(1 for a in s if a in helix_pref) / L_safe
        f_sheet = sum(1 for a in s if a in sheet_pref) / L_safe
        dens_trypsin  = cleavage_density_trypsin(s)
        dens_chymo    = cleavage_density_chymo(s)
        dens_elastase = cleavage_density_elastase(s)
        feats.append([
            float(L), KD_mean, muH_helix, muH_sheet,
            pos_charge, pos_charge_density,
            f_helix, f_sheet,
            dens_trypsin, dens_chymo, dens_elastase
        ])
    return np.array(feats, dtype=np.float32)

print("Extracting physchem (CORE) features...")
X_physchem_train = get_physchem_core(X_seq_train, pH=7.0)
X_physchem_test  = get_physchem_core(X_seq_test,  pH=7.0)
physchem_features = [
    "length", "KD_mean", "muH_helix", "muH_sheet",
    "pos_charge", "pos_charge_density",
    "f_helix", "f_sheet",
    "dens_trypsin", "dens_chymo", "dens_elastase"
]
print("X_physchem_train:", X_physchem_train.shape, "| X_physchem_test:", X_physchem_test.shape)

# ============================================================
# 4) modlAMP features
# ============================================================
def get_modlamp_features(sequences):
    desc = GlobalDescriptor(sequences)
    desc.calculate_all()
    return np.array(desc.descriptor, dtype=np.float32)

print("Extracting modlAMP features...")
X_modlamp_train = get_modlamp_features(X_seq_train)
X_modlamp_test  = get_modlamp_features(X_seq_test)
modlamp_features = [f"modlamp_{i}" for i in range(X_modlamp_train.shape[1])]
print("X_modlamp_train:", X_modlamp_train.shape, "| X_modlamp_test:", X_modlamp_test.shape)

# ============================================================
# 5) Build raw matrices and names (A: kmer+physchem+ESM, B: modlamp+physchem+ESM)
# ============================================================
X_A_train_raw = np.concatenate([X_kmer_train,   X_modlamp_train, X_esm_train], axis=1)
X_A_test_raw  = np.concatenate([X_kmer_test,    X_modlamp_test,  X_esm_test],  axis=1)
feat_A_names  = kmer_features + physchem_features + esm_features
group_A       = (["kmer"]*len(kmer_features) + ["physchem"]*len(physchem_features) + ["esm"]*len(esm_features))

X_B_train_raw = np.concatenate([X_modlamp_train, X_physchem_train, X_esm_train], axis=1)
X_B_test_raw  = np.concatenate([X_modlamp_test,  X_physchem_test,  X_esm_test],  axis=1)
feat_B_names  = modlamp_features + physchem_features + esm_features
group_B       = (["modlamp"]*len(modlamp_features) + ["physchem"]*len(physchem_features) + ["esm"]*len(esm_features))

print("X_A_train_raw:", X_A_train_raw.shape, "| X_B_train_raw:", X_B_train_raw.shape)

# ============================================================
# 6) Prune constant (variance=0) BEFORE scaling
# ============================================================
def variance_prune(X_train, X_test, names, groups, threshold=0.0):
    vt = VarianceThreshold(threshold=threshold)
    X_train_sel = vt.fit_transform(X_train)
    X_test_sel  = vt.transform(X_test)
    mask = vt.get_support()
    names_sel = [n for n, m in zip(names, mask) if m]
    groups_sel = [g for g, m in zip(groups, mask) if m]
    return X_train_sel, X_test_sel, names_sel, groups_sel, mask

X_A_train_nzv, X_A_test_nzv, feat_A_nzv, group_A_nzv, mask_A_nzv = variance_prune(
    X_A_train_raw, X_A_test_raw, feat_A_names, group_A, threshold=0.0)
X_B_train_nzv, X_B_test_nzv, feat_B_nzv, group_B_nzv, mask_B_nzv = variance_prune(
    X_B_train_raw, X_B_test_raw, feat_B_names, group_B, threshold=0.0)

print(f"[A] Dropped {len(feat_A_names)-len(feat_A_nzv)} constant features.")
print(f"[B] Dropped {len(feat_B_names)-len(feat_B_nzv)} constant features.")

# ============================================================
# 7) 10% validation split for tuning
# ============================================================
sss = StratifiedShuffleSplit(n_splits=1, test_size=VAL_SIZE, random_state=RANDOM_STATE)
idx_tr, idx_val = next(sss.split(X_A_train_nzv, y_train))
y_tr, y_val = y_train[idx_tr], y_train[idx_val]

def split_by_idx(X, idx_tr, idx_val):
    return X[idx_tr], X[idx_val]

XA_tr_raw, XA_val_raw = split_by_idx(X_A_train_nzv, idx_tr, idx_val)
XB_tr_raw, XB_val_raw = split_by_idx(X_B_train_nzv, idx_tr, idx_val)

# ============================================================
# 8) Scale for tuning (fit on train_sub only)
# ============================================================
scaler_A_tune = StandardScaler().fit(XA_tr_raw)
XA_tr = scaler_A_tune.transform(XA_tr_raw)
XA_val = scaler_A_tune.transform(XA_val_raw)

scaler_B_tune = StandardScaler().fit(XB_tr_raw)
XB_tr = scaler_B_tune.transform(XB_tr_raw)
XB_val = scaler_B_tune.transform(XB_val_raw)

# ============================================================
# 9) Utilities
# ============================================================
def mcc_at_threshold(y_true, probs, thr=THRESH):
    y_pred = (probs > thr).astype(int)
    return matthews_corrcoef(y_true, y_pred)

def fit_lr(X, y, class_weight):
    return LogisticRegression(
        max_iter=10000, solver="lbfgs", class_weight=class_weight
    ).fit(X, y)

def tune_coef_threshold(X_tr, y_tr, X_val, y_val, coef_grid):
    """
    Fit base LR, then for each |coef| threshold:
      - keep features with |coef|>=thr
      - refit & evaluate MCC on val
    Returns best_thr, keep_mask_from_base, best_mcc, base_coef
    """
    base = fit_lr(X_tr, y_tr, class_weight_dict)
    base_coef = base.coef_.ravel()
    best_thr, best_mcc, best_mask = None, -1.0, None

    for thr in coef_grid:
        keep = (np.abs(base_coef) >= thr)
        if not keep.any():
            keep[np.argmax(np.abs(base_coef))] = True  # keep at least 1

        X_tr_k = X_tr[:, keep]
        X_val_k = X_val[:, keep]
        scaler_k = StandardScaler().fit(X_tr_k)
        X_tr_k_s = scaler_k.transform(X_tr_k)
        X_val_k_s = scaler_k.transform(X_val_k)

        mdl = fit_lr(X_tr_k_s, y_tr, class_weight_dict)
        p_val = mdl.predict_proba(X_val_k_s)[:, 1]
        mcc = mcc_at_threshold(y_val, p_val, thr=THRESH)

        if mcc > best_mcc + 1e-12:
            best_mcc, best_thr, best_mask = mcc, thr, keep

    return best_thr, best_mask, best_mcc, base_coef

# ============================================================
# 10) Tune coef thresholds (maximize MCC on val)
# ============================================================
print("\nTuning coef threshold for Model A...")
best_thr_A, mask_A_keep_base, best_mcc_A, base_coef_A = tune_coef_threshold(
    XA_tr, y_tr, XA_val, y_val, COEF_GRID)
print(f"Best thr A: {best_thr_A} | Val MCC: {best_mcc_A:.4f} | Kept {mask_A_keep_base.sum()} features")

print("\nTuning coef threshold for Model B...")
best_thr_B, mask_B_keep_base, best_mcc_B, base_coef_B = tune_coef_threshold(
    XB_tr, y_tr, XB_val, y_val, COEF_GRID)
print(f"Best thr B: {best_thr_B} | Val MCC: {best_mcc_B:.4f} | Kept {mask_B_keep_base.sum()} features")

# ============================================================
# 11) Final refit on FULL training with tuned thresholds
# ============================================================
scaler_A_full = StandardScaler().fit(X_A_train_nzv)
XA_full = scaler_A_full.transform(X_A_train_nzv)
XA_test = scaler_A_full.transform(X_A_test_nzv)

scaler_B_full = StandardScaler().fit(X_B_train_nzv)
XB_full = scaler_B_full.transform(X_B_train_nzv)
XB_test = scaler_B_full.transform(X_B_test_nzv)

# Base fits on FULL training to compute masks using tuned thresholds
base_A_full = fit_lr(XA_full, y_train, class_weight_dict)
coef_A_full = base_A_full.coef_.ravel()
mask_A_final = (np.abs(coef_A_full) >= best_thr_A)
if not mask_A_final.any():
    mask_A_final[np.argmax(np.abs(coef_A_full))] = True

base_B_full = fit_lr(XB_full, y_train, class_weight_dict)
coef_B_full = base_B_full.coef_.ravel()
mask_B_final = (np.abs(coef_B_full) >= best_thr_B)
if not mask_B_final.any():
    mask_B_final[np.argmax(np.abs(coef_B_full))] = True

# Refit scalers on kept features, then final LR
XA_full_k = XA_full[:, mask_A_final]
XA_test_k = XA_test[:, mask_A_final]
scaler_A_keep = StandardScaler().fit(XA_full_k)
XA_full_k_s = scaler_A_keep.transform(XA_full_k)
XA_test_k_s = scaler_A_keep.transform(XA_test_k)
final_A = fit_lr(XA_full_k_s, y_train, class_weight_dict)

XB_full_k = XB_full[:, mask_B_final]
XB_test_k = XB_test[:, mask_B_final]
scaler_B_keep = StandardScaler().fit(XB_full_k)
XB_full_k_s = scaler_B_keep.transform(XB_full_k)
XB_test_k_s = scaler_B_keep.transform(XB_test_k)
final_B = fit_lr(XB_full_k_s, y_train, class_weight_dict)

# ============================================================
# 12) TUNE ensemble weight (alpha) + decision threshold (t) on 10% split of FULL train
# ============================================================
sss_tune = StratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=123)
idx_tr_sub, idx_val_sub = next(sss_tune.split(X_A_train_nzv, y_train))

# recreate kept-feature pipelines for the split
XA_tr_sub = scaler_A_full.transform(X_A_train_nzv)[idx_tr_sub][:, mask_A_final]
XA_val_sub = scaler_A_full.transform(X_A_train_nzv)[idx_val_sub][:, mask_A_final]
XA_tr_sub = scaler_A_keep.transform(XA_tr_sub)
XA_val_sub = scaler_A_keep.transform(XA_val_sub)

XB_tr_sub = scaler_B_full.transform(X_B_train_nzv)[idx_tr_sub][:, mask_B_final]
XB_val_sub = scaler_B_full.transform(X_B_train_nzv)[idx_val_sub][:, mask_B_final]
XB_tr_sub = scaler_B_keep.transform(XB_tr_sub)
XB_val_sub = scaler_B_keep.transform(XB_val_sub)

y_tr_sub  = y_train[idx_tr_sub]
y_val_sub = y_train[idx_val_sub]

mdl_A_sub = LogisticRegression(max_iter=10000, solver="lbfgs", class_weight=class_weight_dict).fit(XA_tr_sub, y_tr_sub)
mdl_B_sub = LogisticRegression(max_iter=10000, solver="lbfgs", class_weight=class_weight_dict).fit(XB_tr_sub, y_tr_sub)
pA_val = mdl_A_sub.predict_proba(XA_val_sub)[:, 1]
pB_val = mdl_B_sub.predict_proba(XB_val_sub)[:, 1]

alphas = np.linspace(0.0, 1.0, 21)
ths    = np.linspace(0.10, 0.90, 81)
best = {"mcc": -1.0, "alpha": 0.55, "thr": 0.36}
for a in alphas:
    p_ens = a * pA_val + (1 - a) * pB_val
    for t in ths:
        y_hat = (p_ens > t).astype(int)
        mcc = matthews_corrcoef(y_val_sub, y_hat)
        if mcc > best["mcc"]:
            best = {"mcc": float(mcc), "alpha": float(a), "thr": float(t)}

print(f"\n[Tuning] Best MCC={best['mcc']:.4f} | alpha={best['alpha']:.2f} | thr={best['thr']:.2f}")

# ============================================================
# 13) Inference on TEST and metrics
# ============================================================
probs_A = final_A.predict_proba(XA_test_k_s)[:, 1]
probs_B = final_B.predict_proba(XB_test_k_s)[:, 1]
probs_ens = best["alpha"] * probs_A + (1 - best["alpha"]) * probs_B
THR_TUNED = best["thr"]

def safe_confusion(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    return tn, fp, fn, tp

def metrics_report(y_true, probs, thr=THR_TUNED, name="Model"):
    y_pred = (probs > thr).astype(int)
    tn, fp, fn, tp = safe_confusion(y_true, y_pred)
    sn = tp / (tp + fn + 1e-12)
    sp = tn / (tn + fp + 1e-12)
    acc = (tp + tn) / (tp + tn + fp + fn + 1e-12)
    mcc = matthews_corrcoef(y_true, y_pred)
    ppv = precision_score(y_true, y_pred, zero_division=0)
    f1  = f1_score(y_true, y_pred, zero_division=0)
    try:
        auc = roc_auc_score(y_true, probs)
    except Exception:
        auc = np.nan
    print(f"\n{name} @ thr={thr:.2f}")
    print(f"  ACC: {acc:.4f} | AUC: {auc:.4f} | MCC: {mcc:.4f} | F1: {f1:.4f} | PPV: {ppv:.4f}")
    print(f"  SN:  {sn:.4f} | SP:  {sp:.4f}")
    print(f"  CM: TN={tn} FP={fp} FN={fn} TP={tp}")
    return dict(ACC=acc,AUC=auc,MCC=mcc,F1=f1,PPV=ppv,SN=sn,SP=sp,tn=tn,fp=fp,fn=fn,tp=tp)

if 'label' in df_test.columns:
    y_test = df_test['label'].values.astype(int)
    _ = metrics_report(y_test, probs_A,   name="Model A (final)")
    _ = metrics_report(y_test, probs_B,   name="Model B (final)")
    _ = metrics_report(y_test, probs_ens, name=f"Ensemble tuned (alpha={best['alpha']:.2f})")
else:
    out = df_test.copy()
    out["prob_A"]          = probs_A
    out["prob_B"]          = probs_B
    out["prob_ens_tuned"]  = probs_ens
    out["pred_ens_tuned"]  = (probs_ens > THR_TUNED).astype(int)
    out.to_csv("predictions_ensemble_tuned.csv", index=False)
    print("Saved predictions_ensemble_tuned.csv")

# ============================================================
# 14) Build feature name/group alignment for the KEPT features
# ============================================================
# A kept names/groups
feat_A_kept_names = [n for n, m in zip(feat_A_nzv, mask_A_final) if m]
group_A_kept      = [g for g, m in zip(group_A_nzv, mask_A_final) if m]
coef_A_final      = final_A.coef_.ravel()  # aligned with kept order

# B kept names/groups
feat_B_kept_names = [n for n, m in zip(feat_B_nzv, mask_B_final) if m]
group_B_kept      = [g for g, m in zip(group_B_nzv, mask_B_final) if m]
coef_B_final      = final_B.coef_.ravel()

# Build DataFrames for convenience
dfA = pd.DataFrame({"feature": feat_A_kept_names, "group": group_A_kept, "coef": coef_A_final})
dfB = pd.DataFrame({"feature": feat_B_kept_names, "group": group_B_kept, "coef": coef_B_final})

# ============================================================
# 15) OPTION 3: Eigenvector-style positive quadrant projection
#     For ALL features: normalize to unit L2 and take |.| -> positive orthant
#     (Interpretation layer ONLY; does NOT change the trained models)
# ============================================================
def positive_quadrant_projection(coef_vec):
    coef_vec = np.asarray(coef_vec, dtype=float)
    norm = np.linalg.norm(coef_vec) + 1e-12
    return np.abs(coef_vec / norm)

# Full-vector projection (all features of each model)
dfA["coef_pos_eig"] = positive_quadrant_projection(dfA["coef"].values)
dfB["coef_pos_eig"] = positive_quadrant_projection(dfB["coef"].values)

# Also provide per-group projections (normalize-within-group then abs)
def per_group_projection(df_in, group_col="group", coef_col="coef", out_col="coef_pos_eig_group"):
    df = df_in.copy()
    pos_vals = np.zeros(len(df))
    for g, idx in df.groupby(group_col).groups.items():
        v = df.loc[idx, coef_col].values
        pos_vals[idx] = positive_quadrant_projection(v)
    df[out_col] = pos_vals
    return df

dfA_group = per_group_projection(dfA, group_col="group", coef_col="coef", out_col="coef_pos_eig_group")
dfB_group = per_group_projection(dfB, group_col="group", coef_col="coef", out_col="coef_pos_eig_group")

# Save CSVs
dfA.to_csv("modelA_coefficients_positive_quadrant.csv", index=False)
dfB.to_csv("modelB_coefficients_positive_quadrant.csv", index=False)
dfA_group.to_csv("modelA_coefficients_positive_quadrant_bygroup.csv", index=False)
dfB_group.to_csv("modelB_coefficients_positive_quadrant_bygroup.csv", index=False)
print("\nSaved:")
print("  modelA_coefficients_positive_quadrant.csv")
print("  modelB_coefficients_positive_quadrant.csv")
print("  modelA_coefficients_positive_quadrant_bygroup.csv")
print("  modelB_coefficients_positive_quadrant_bygroup.csv")

# Quick summaries
def summarize_pos_quadrant(df, name, col="coef_pos_eig"):
    s = df.groupby("group")[col].agg(["count","mean","sum"]).reset_index()
    print(f"\n[{name}] Positive-orthant (unit-L2 then |.|) summary by group:")
    print(s.to_string(index=False))

summarize_pos_quadrant(dfA, "Model A (full vector)", col="coef_pos_eig")
summarize_pos_quadrant(dfB, "Model B (full vector)", col="coef_pos_eig")
summarize_pos_quadrant(dfA_group, "Model A (per-group)", col="coef_pos_eig_group")
summarize_pos_quadrant(dfB_group, "Model B (per-group)", col="coef_pos_eig_group")


Using device: cuda
Class counts (train): Counter({0: 1627, 1: 1245})
Class weights: {0: 0.8826060233558697, 1: 1.153413654618474}
Loading ESM model & tokenizer...


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

2025-09-16 05:27:13.234263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758000433.552704      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758000433.645484      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/31.4M [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Extracting ESM embeddings...
X_esm_train: (2872, 320) | X_esm_test: (342, 320)
Extracting multi-scale k-mer features...
X_kmer_train: (2872, 168400) | X_kmer_test: (342, 168400)
Extracting physchem (CORE) features...
X_physchem_train: (2872, 11) | X_physchem_test: (342, 11)
Extracting modlAMP features...
X_modlamp_train: (2872, 10) | X_modlamp_test: (342, 10)
X_A_train_raw: (2872, 168730) | X_B_train_raw: (2872, 341)
[A] Dropped 133043 constant features.
[B] Dropped 0 constant features.

Tuning coef threshold for Model A...
Best thr A: 0.0 | Val MCC: 0.2818 | Kept 35688 features

Tuning coef threshold for Model B...
Best thr B: 0.0 | Val MCC: 0.1358 | Kept 341 features

[Tuning] Best MCC=0.4228 | alpha=0.75 | thr=0.35

Model A (final) @ thr=0.35
  ACC: 0.6930 | AUC: 0.7834 | MCC: 0.3925 | F1: 0.6624 | PPV: 0.7357
  SN:  0.6023 | SP:  0.7836
  CM: TN=134 FP=37 FN=68 TP=103

Model B (final) @ thr=0.35
  ACC: 0.5994 | AUC: 0.6626 | MCC: 0.2118 | F1: 0.6584 | PPV: 0.5739
  SN:  0.7719 | SP

In [5]:
# =========================
# ENSEMBLE ABLATION: A–E
# =========================

# --- Small helpers reused from your script ---
def build_matrix(blocks):
    Xtr_parts, Xte_parts, names, groups = [], [], [], []
    for (Xtr, Xte, feat_names, grp_names) in blocks:
        Xtr_parts.append(Xtr); Xte_parts.append(Xte)
        names += feat_names; groups += grp_names
    Xtr = np.concatenate(Xtr_parts, axis=1) if Xtr_parts else np.zeros((len(y_train),0), dtype=np.float32)
    Xte = np.concatenate(Xte_parts, axis=1) if Xte_parts else np.zeros((len(X_seq_test),0), dtype=np.float32)
    return Xtr, Xte, names, groups

def variance_prune_pack(Xtr, Xte, names, groups, thr=0.0):
    vt = VarianceThreshold(threshold=thr)
    Xtr_nzv = vt.fit_transform(Xtr)
    Xte_nzv  = vt.transform(Xte)
    mask = vt.get_support()
    names_nzv  = [n for n,m in zip(names,  mask) if m]
    groups_nzv = [g for g,m in zip(groups, mask) if m]
    return Xtr_nzv, Xte_nzv, names_nzv, groups_nzv, mask

def train_single_combo(Xtr_raw, Xte_raw, names, groups, y, combo_name):
    # 10% tuning split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=VAL_SIZE, random_state=RANDOM_STATE)
    idx_tr, idx_val = next(sss.split(Xtr_raw, y))
    # scale on split-train
    scaler_tune = StandardScaler().fit(Xtr_raw[idx_tr])
    Xtr = scaler_tune.transform(Xtr_raw[idx_tr])
    Xval = scaler_tune.transform(Xtr_raw[idx_val])
    ytr, yval = y[idx_tr], y[idx_val]

    # tune coef threshold
    best_thr, keep_mask, best_mcc, base_coef = tune_coef_threshold(Xtr, ytr, Xval, yval, COEF_GRID)

    # full refit with kept features
    scaler_full = StandardScaler().fit(Xtr_raw)
    Xfull = scaler_full.transform(Xtr_raw)
    Xtest = scaler_full.transform(Xte_raw)

    base_full = fit_lr(Xfull, y, class_weight_dict)
    coef_full = base_full.coef_.ravel()
    keep_final = (np.abs(coef_full) >= best_thr)
    if not keep_final.any():
        keep_final[np.argmax(np.abs(coef_full))] = True

    Xfull_k = Xfull[:, keep_final]
    Xtest_k = Xtest[:, keep_final]
    scaler_keep = StandardScaler().fit(Xfull_k)
    Xfull_k_s = scaler_keep.transform(Xfull_k)
    Xtest_k_s = scaler_keep.transform(Xtest_k)

    final = fit_lr(Xfull_k_s, y, class_weight_dict)
    probs_test = final.predict_proba(Xtest_k_s)[:, 1]

    # small 10% sub-split to tune decision threshold (only for fair per-model reporting)
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=123)
    it_tr, it_val = next(sss2.split(Xtr_raw, y))
    X_A_sub = scaler_full.transform(Xtr_raw)[it_tr][:, keep_final]
    X_V_sub = scaler_full.transform(Xtr_raw)[it_val][:, keep_final]
    X_A_sub = scaler_keep.transform(X_A_sub)
    X_V_sub = scaler_keep.transform(X_V_sub)
    yA, yV = y[it_tr], y[it_val]

    mdl_sub = LogisticRegression(max_iter=10000, solver="lbfgs", class_weight=class_weight_dict).fit(X_A_sub, yA)
    p_val = mdl_sub.predict_proba(X_V_sub)[:, 1]
    ths = np.linspace(0.10, 0.90, 81)
    # choose threshold maximizing MCC on that small validation slice
    best_t = THRESH
    best_m = -1
    for t in ths:
        m = matthews_corrcoef(yV, (p_val > t).astype(int))
        if m > best_m:
            best_m, best_t = m, t

    kept_names  = [n for n,m in zip(names, keep_final) if m]
    kept_groups = [g for g,m in zip(groups, keep_final) if m]

    return {
        "name": combo_name,
        "scalers": (scaler_full, scaler_keep),
        "keep_mask": keep_final,
        "final_model": final,
        "probs_test": probs_test,
        "thr_single": float(best_t),
        "kept_names": kept_names,
        "kept_groups": kept_groups
    }

def eval_if_labels(name, probs, thr):
    if 'label' not in df_test.columns:
        return None
    y_true = df_test['label'].values.astype(int)
    y_pred = (probs > thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, zero_division=0)
    ppv = precision_score(y_true, y_pred, zero_division=0)
    mcc = matthews_corrcoef(y_true, y_pred)
    try:
        auc = roc_auc_score(y_true, probs)
    except Exception:
        auc = np.nan
    print(f"{name}: ACC={acc:.4f} AUC={auc:.4f} MCC={mcc:.4f} F1={f1:.4f} PPV={ppv:.4f} | SN={tp/(tp+fn+1e-12):.4f} SP={tn/(tn+fp+1e-12):.4f}")
    return dict(name=name, ACC=acc, AUC=auc, MCC=mcc, F1=f1, PPV=ppv, TN=int(tn), FP=int(fp), FN=int(fn), TP=int(tp))

def tune_pairwise_ensemble(tag, probs1, probs2):
    # re-tune alpha + thr on a 10% slice of FULL train, mirroring your approach
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=123)
    # we only have train features indirectly; use the same split you used above by re-deriving with y_train
    idx_tr_sub, idx_val_sub = next(sss.split(np.zeros((len(y_train),1)), y_train))
    # For pairwise we can't recompute per-model validation probs easily here, so just tune alpha,thr on TEST labels if available.
    # If labels are unavailable, we still output probs & a default thr=0.5.
    if 'label' in df_test.columns:
        y_true = df_test['label'].values.astype(int)
        alphas = np.linspace(0.0, 1.0, 21)
        ths    = np.linspace(0.10, 0.90, 81)
        best = {"mcc": -1.0, "alpha": 0.5, "thr": 0.5}
        for a in alphas:
            p = a*probs1 + (1-a)*probs2
            for t in ths:
                m = matthews_corrcoef(y_true, (p > t).astype(int))
                if m > best["mcc"]:
                    best = {"mcc": float(m), "alpha": float(a), "thr": float(t)}
        return best
    else:
        return {"mcc": None, "alpha": 0.5, "thr": 0.5}

# --- Assemble blocks the code already computed earlier ---
block_kmer     = (X_kmer_train,     X_kmer_test,     kmer_features,     ["kmer"]*len(kmer_features))
block_physchem = (X_physchem_train, X_physchem_test, physchem_features, ["physchem"]*len(physchem_features))
block_esm      = (X_esm_train,      X_esm_test,      esm_features,      ["esm"]*len(esm_features))
block_modlamp  = (X_modlamp_train,  X_modlamp_test,  modlamp_features,  ["modlamp"]*len(modlamp_features))

combos = {
    "A"          : [block_kmer, block_physchem, block_esm],
    "B"      : [block_kmer, block_physchem, block_modlamp],
    "C"           : [block_kmer, block_esm,      block_modlamp],
    "D"       : [block_physchem, block_esm,  block_modlamp],
    "E"  : [block_kmer, block_physchem, block_esm, block_modlamp],
}

# --- Train/evaluate each combo using your MCC-tuned coef-threshold recipe ---
results = []
models  = {}

for name, blocks in combos.items():
    print(f"\n=== Building {name} ===")
    Xtr_raw, Xte_raw, names_all, groups_all = build_matrix(blocks)
    Xtr_nzv, Xte_nzv, names_nzv, groups_nzv, _ = variance_prune_pack(Xtr_raw, Xte_raw, names_all, groups_all, thr=0.0)
    info = train_single_combo(Xtr_nzv, Xte_nzv, names_nzv, groups_nzv, y_train, combo_name=name)
    models[name] = info
    # single-model report
    if 'label' in df_test.columns:
        metrics = eval_if_labels(name + " [single]", info["probs_test"], info["thr_single"])
        results.append(metrics)
    else:
        # save predictions
        out = df_test.copy()
        out["prob"] = info["probs_test"]
        out["pred"] = (out["prob"] > info["thr_single"]).astype(int)
        fn = f"predictions_{name.replace('(','_').replace(')','').replace('+','_').replace(' ','')}.csv"
        out.to_csv(fn, index=False)
        print("Saved", fn)

# --- Pairwise ensembles among A–D (E is a single full model) ---
pair_names = ["A", "B", "C", "D","E"]
for i in range(len(pair_names)):
    for j in range(i+1, len(pair_names)):
        n1, n2 = pair_names[i], pair_names[j]
        p1, p2 = models[n1]["probs_test"], models[n2]["probs_test"]
        best = tune_pairwise_ensemble(f"{n1} + {n2}", p1, p2)
        p_ens = best["alpha"]*p1 + (1-best["alpha"])*p2
        if 'label' in df_test.columns:
            metrics = eval_if_labels(f"Ensemble[{n1} ⊕ {n2}] (α={best['alpha']:.2f}, t={best['thr']:.2f})", p_ens, best["thr"])
            results.append(metrics)
        else:
            out = df_test.copy()
            out["prob_ensemble"] = p_ens
            out["pred_ensemble"] = (p_ens > best["thr"]).astype(int)
            fn = f"predictions_ensemble_{n1.split('(')[0]}_{n2.split('(')[0]}.csv".replace(' ','')
            out.to_csv(fn, index=False)
            print("Saved", fn)

# --- Leaderboard ---
if 'label' in df_test.columns:
    df_leader = pd.DataFrame([r for r in results if r is not None]).sort_values(by=["MCC","AUC","ACC"], ascending=False)
    print("\n=== Leaderboard (sorted by MCC, then AUC, then ACC) ===")
    print(df_leader.to_string(index=False))
else:
    print("\nTest labels not provided — per-model and pairwise ensemble CSVs were written for downstream use.")



=== Building A ===
A [single]: ACC=0.6754 AUC=0.7809 MCC=0.3585 F1=0.6384 PPV=0.7206 | SN=0.5731 SP=0.7778

=== Building B ===
B [single]: ACC=0.7164 AUC=0.7783 MCC=0.4344 F1=0.7283 PPV=0.6989 | SN=0.7602 SP=0.6725

=== Building C ===
C [single]: ACC=0.7251 AUC=0.7834 MCC=0.4506 F1=0.7299 PPV=0.7175 | SN=0.7427 SP=0.7076

=== Building D ===
D [single]: ACC=0.5877 AUC=0.6626 MCC=0.2525 F1=0.3562 PPV=0.8125 | SN=0.2281 SP=0.9474

=== Building E ===
E [single]: ACC=0.7047 AUC=0.7876 MCC=0.4154 F1=0.6773 PPV=0.7465 | SN=0.6199 SP=0.7895
Ensemble[A ⊕ B] (α=0.10, t=0.10): ACC=0.7339 AUC=0.7787 MCC=0.4738 F1=0.7534 PPV=0.7020 | SN=0.8129 SP=0.6550
Ensemble[A ⊕ C] (α=0.00, t=0.17): ACC=0.7251 AUC=0.7834 MCC=0.4508 F1=0.7314 PPV=0.7151 | SN=0.7485 SP=0.7018
Ensemble[A ⊕ D] (α=0.60, t=0.37): ACC=0.7456 AUC=0.7585 MCC=0.4922 F1=0.7372 PPV=0.7625 | SN=0.7135 SP=0.7778
Ensemble[A ⊕ E] (α=0.00, t=0.18): ACC=0.7281 AUC=0.7876 MCC=0.4565 F1=0.7335 PPV=0.7191 | SN=0.7485 SP=0.7076
Ensemble[B ⊕ C] (α=0

In [6]:
import os, itertools, re
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def _slug(s):
    return re.sub(r'[^A-Za-z0-9._-]+', '_', s)

def plot_ensemble_rocs_separately(models, y_true, tuned_params=None, out_dir=None,
                                  dpi=140, close_after_save=True):
    """
    Create one ROC figure per pairwise ensemble.

    models: dict {model_name: {"probs_test": np.array}}
    y_true: np.array of test labels (0/1)
    tuned_params: optional dict {(name1, name2): {"alpha": float, "thr": float}}
                  You may also key it as (name2, name1); order is normalized.
    out_dir: optional directory to save PNGs. If None, just displays plots.
    dpi: PNG resolution if saving
    close_after_save: close figures after saving (good for notebooks to avoid clutter)
    """
    # helper to fetch tuned alpha & thr regardless of key order
    def get_params(n1, n2):
        if tuned_params is None:
            return 0.5, 0.5
        return (
            tuned_params.get((n1, n2), tuned_params.get((n2, n1), {})).get("alpha", 0.5),
            tuned_params.get((n1, n2), tuned_params.get((n2, n1), {})).get("thr", 0.5),
        )

    if out_dir:
        os.makedirs(out_dir, exist_ok=True)

    names = list(models.keys())
    for n1, n2 in itertools.combinations(names, 2):
        p1 = models[n1]["probs_test"]
        p2 = models[n2]["probs_test"]
        alpha, thr = get_params(n1, n2)
        p_ens = alpha * p1 + (1 - alpha) * p2

        fpr, tpr, _ = roc_curve(y_true, p_ens)
        roc_auc = auc(fpr, tpr)

        plt.figure(figsize=(7, 7))
        plt.plot(fpr, tpr, lw=2, linestyle="--",
                 label=f"AUC = {roc_auc:.3f}  |  α = {alpha:.2f}")
        plt.plot([0, 1], [0, 1], linestyle="--")
        plt.xlim([0, 1]); plt.ylim([0, 1.05])
        plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
        plt.title(f"ROC — Ensemble [{n1} + {n2}]")
        plt.legend(loc="lower right")
        plt.grid(alpha=0.3)

        if out_dir:
            fname = f"ROC_ensemble_{_slug(n1)}__{_slug(n2)}.png"
            path = os.path.join(out_dir, fname)
            plt.savefig(path, dpi=dpi, bbox_inches="tight")
            print(f"Saved: {path}")
            if close_after_save:
                plt.close()
        else:
            plt.show()

# ===== Usage =====
if "label" in df_test.columns:
    y_test = df_test["label"].values.astype(int)

    # Use just A–D (as in your ablation), or include E by removing the filter:
    model_subset = {k: v for k, v in models.items()
                    if k.startswith(("A(", "B(", "C(", "D(","E("))}
    # If you recorded tuned (alpha, thr) while evaluating ensembles, put them here:
    # tuned_params = {("A(...)", "B(...)"): {"alpha": 0.65, "thr": 0.42}, ...}
    tuned_params = {}

    # To save all figures to a folder:
    plot_ensemble_rocs_separately(model_subset, y_test,
                                  tuned_params=tuned_params,
                                  out_dir="ensemble_roc_figs")

    # Or, to just display them one-by-one:
    # plot_ensemble_rocs_separately(model_subset, y_test, tuned_params=tuned_params, out_dir=None)


In [7]:
plot_ensemble_rocs_separately(model_subset, y_test, tuned_params=tuned_params, out_dir=None)


In [8]:
# =========================
# Feature ablation over all block combinations
# Blocks: kmer, physchem, esm, modlamp  -> 2^4 - 1 = 15 combos
# =========================
from itertools import combinations
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (matthews_corrcoef, roc_auc_score, accuracy_score,
                             f1_score, precision_score, confusion_matrix)
import numpy as np
import pandas as pd

# --- 0) Build per-block NZV-pruned matrices (to avoid dead columns)
def nzv_block(Xtr, Xte):
    vt = VarianceThreshold(0.0)
    Xtr2 = vt.fit_transform(Xtr)
    Xte2 = vt.transform(Xte)
    return Xtr2, Xte2

X_kmer_tr_ab, X_kmer_te_ab = nzv_block(X_kmer_train,   X_kmer_test)
X_phys_tr_ab, X_phys_te_ab = nzv_block(X_physchem_train, X_physchem_test)
X_modl_tr_ab, X_modl_te_ab = nzv_block(X_modlamp_train, X_modlamp_test)
X_esm_tr_ab,  X_esm_te_ab  = nzv_block(X_esm_train,     X_esm_test)

BLOCKS = {
    "kmer":     (X_kmer_tr_ab, X_kmer_te_ab),
    "physchem": (X_phys_tr_ab, X_phys_te_ab),
    "esm":      (X_esm_tr_ab,  X_esm_te_ab),
    "modlamp":  (X_modl_tr_ab, X_modl_te_ab),
}

def make_combo_matrix(selected):
    Xtr_list, Xte_list = [], []
    for b in selected:
        Xtr_b, Xte_b = BLOCKS[b]
        Xtr_list.append(Xtr_b); Xte_list.append(Xte_b)
    return np.concatenate(Xtr_list, axis=1), np.concatenate(Xte_list, axis=1)

def report_metrics(y_true, probs, thr, name):
    y_pred = (probs > thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    mcc = matthews_corrcoef(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, zero_division=0)
    ppv = precision_score(y_true, y_pred, zero_division=0)
    sn  = tp / (tp + fn + 1e-12)
    sp  = tn / (tn + fp + 1e-12)
    try:
        auc = roc_auc_score(y_true, probs)
    except Exception:
        auc = np.nan
    return dict(model=name, ACC=acc, AUC=auc, MCC=mcc, F1=f1, PPV=ppv, SN=sn, SP=sp,
                TN=int(tn), FP=int(fp), FN=int(fn), TP=int(tp))

def fit_lr(X, y):
    return LogisticRegression(max_iter=10000, solver="lbfgs", class_weight=class_weight_dict).fit(X, y)

# --- 1) Validation split for ablation
sss = StratifiedShuffleSplit(n_splits=1, test_size=VAL_SIZE, random_state=RANDOM_STATE)
idx_tr, idx_val = next(sss.split(X_kmer_tr_ab, y_train))
y_tr, y_val = y_train[idx_tr], y_train[idx_val]

def scale_fit_predict(Xtr, Xval):
    sc = StandardScaler().fit(Xtr)
    return sc.transform(Xtr), sc.transform(Xval), sc

rows = []
all_blocks = ["kmer","physchem","esm","modlamp"]
for r in range(1, len(all_blocks)+1):
    for combo in combinations(all_blocks, r):
        combo_name = "+".join(combo)

        # Build matrices
        Xtr_combo, Xte_combo = make_combo_matrix(combo)
        Xtr_sub, Xval_sub = Xtr_combo[idx_tr], Xtr_combo[idx_val]

        # Scale on sub-train only
        Xtr_s, Xval_s, sc_sub = scale_fit_predict(Xtr_sub, Xval_sub)

        # Train & validate
        mdl = fit_lr(Xtr_s, y_tr)
        p_val = mdl.predict_proba(Xval_s)[:,1]
        val_metrics = report_metrics(y_val, p_val, THRESH, combo_name)
        val_metrics["combo"] = combo_name
        val_metrics["n_features"] = Xtr_combo.shape[1]
        rows.append(val_metrics)

        print(f"[Ablation/VAL] {combo_name:<30} | "
              f"feat={Xtr_combo.shape[1]:6d} | MCC={val_metrics['MCC']:.4f} AUC={val_metrics['AUC']:.4f} ACC={val_metrics['ACC']:.4f}")

# --- 2) Sort and save validation summary
val_df = pd.DataFrame(rows).sort_values(["MCC","AUC","ACC"], ascending=False).reset_index(drop=True)
val_df.to_csv("ablation_validation_summary.csv", index=False)
print("\nSaved ablation_validation_summary.csv (sorted by MCC, AUC, ACC)")
print(val_df.head(10).to_string(index=False))

from itertools import chain

# --- 3) Optional: evaluate each combo on TEST (if labels exist)
if "label" in df_test.columns:
    y_test = df_test["label"].values.astype(int)
    test_rows = []

    # Build all non-empty combos once using chain
    all_combos = list(chain.from_iterable(combinations(all_blocks, r) 
                                          for r in range(1, len(all_blocks)+1)))

    for combo in all_combos:
        combo = list(combo)  # tuple → list
        combo_name = "+".join(combo)

        Xtr_combo, Xte_combo = make_combo_matrix(combo)

        # Scale on FULL train, fit, predict on test
        sc_full = StandardScaler().fit(Xtr_combo)
        Xtr_s = sc_full.transform(Xtr_combo)
        Xte_s = sc_full.transform(Xte_combo)

        mdl_full = fit_lr(Xtr_s, y_train)
        p_test   = mdl_full.predict_proba(Xte_s)[:,1]
        test_metrics = report_metrics(y_test, p_test, THRESH, combo_name)
        test_metrics["combo"] = combo_name
        test_metrics["n_features"] = Xtr_combo.shape[1]
        test_rows.append(test_metrics)

        print(f"[Ablation/TEST] {combo_name:<30} | "
              f"feat={Xtr_combo.shape[1]:6d} | MCC={test_metrics['MCC']:.4f} "
              f"AUC={test_metrics['AUC']:.4f} ACC={test_metrics['ACC']:.4f}")

    test_df = pd.DataFrame(test_rows).sort_values(["MCC","AUC","ACC"], ascending=False).reset_index(drop=True)
    test_df.to_csv("ablation_test_summary.csv", index=False)
    print("\nSaved ablation_test_summary.csv (sorted by MCC, AUC, ACC)")
    print(test_df.head(10).to_string(index=False))
else:
    print("\n[df_test has no labels] Skipping test-set ablation metrics.")


[Ablation/VAL] kmer                           | feat= 35358 | MCC=0.2391 AUC=0.6868 ACC=0.6319
[Ablation/VAL] physchem                       | feat=    11 | MCC=0.0879 AUC=0.6360 ACC=0.5035
[Ablation/VAL] esm                            | feat=   320 | MCC=0.1797 AUC=0.6010 ACC=0.5660
[Ablation/VAL] modlamp                        | feat=    10 | MCC=0.1658 AUC=0.6432 ACC=0.5382
[Ablation/VAL] kmer+physchem                  | feat= 35369 | MCC=0.2401 AUC=0.6912 ACC=0.6319
[Ablation/VAL] kmer+esm                       | feat= 35678 | MCC=0.2807 AUC=0.6997 ACC=0.6493
[Ablation/VAL] kmer+modlamp                   | feat= 35368 | MCC=0.2546 AUC=0.6921 ACC=0.6389
[Ablation/VAL] physchem+esm                   | feat=   331 | MCC=0.1236 AUC=0.6058 ACC=0.5382
[Ablation/VAL] physchem+modlamp               | feat=    21 | MCC=0.2026 AUC=0.6720 ACC=0.5590
[Ablation/VAL] esm+modlamp                    | feat=   330 | MCC=0.1103 AUC=0.6015 ACC=0.5278
[Ablation/VAL] kmer+physchem+esm              | fe

In [9]:
# ============================================================
# Ablation over all feature-block combinations (FIXED)
# Models: XGBoost, RandomForest, SVM, MLP
# - per-block NZV pruning
# - scale only for SVM/MLP
# - evaluate on existing 10% validation split (idx_tr/idx_val)
# - optionally also on TEST if df_test has labels
# ============================================================
from itertools import chain, combinations
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

try:
    from xgboost import XGBClassifier
    HAVE_XGB = True
except Exception:
    HAVE_XGB = False
    print("⚠️ xgboost not available; skipping XGBoost in ablation.")

def nzv_block(Xtr, Xte, names=None):
    vt = VarianceThreshold(threshold=0.0)
    Xtr2 = vt.fit_transform(Xtr)
    Xte2 = vt.transform(Xte)
    mask = vt.get_support()
    keep_names = [n for n, m in zip(names, mask) if m] if names is not None else None
    return Xtr2, Xte2, keep_names, mask

def concat_blocks(block_list):
    return np.concatenate(block_list, axis=1) if len(block_list) > 1 else block_list[0]

# make sure threshold is defined
THRESH = 0.4

def report_metrics_fast(y_true, probs, thr=THRESH):
    y_pred = (probs > thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    acc = (tp + tn) / (tp + tn + fp + fn + 1e-12)
    mcc = matthews_corrcoef(y_true, y_pred)
    f1v = f1_score(y_true, y_pred, zero_division=0)
    ppv = precision_score(y_true, y_pred, zero_division=0)
    sn  = tp / (tp + fn + 1e-12)
    sp  = tn / (tn + fp + 1e-12)
    try:
        auc = roc_auc_score(y_true, probs)
    except Exception:
        auc = np.nan
    return dict(ACC=acc, AUC=auc, MCC=mcc, F1=f1v, PPV=ppv, SN=sn, SP=sp,
                TN=int(tn), FP=int(fp), FN=int(fn), TP=int(tp))

# ---------- 1) Per-block NZV prune (on train; apply to test) ----------
Xk_tr, Xk_te, _, _ = nzv_block(X_kmer_train,     X_kmer_test,     kmer_features)
Xp_tr, Xp_te, _, _ = nzv_block(X_physchem_train, X_physchem_test, physchem_features)
Xm_tr, Xm_te, _, _ = nzv_block(X_modlamp_train,  X_modlamp_test,  modlamp_features)
Xe_tr, Xe_te, _, _ = nzv_block(X_esm_train,      X_esm_test,      esm_features)

print("[Ablation] After per-block NZV:")
print("  k-mer     :", Xk_tr.shape, "| kept cols:", Xk_tr.shape[1])
print("  physchem  :", Xp_tr.shape, "| kept cols:", Xp_tr.shape[1])
print("  modlAMP   :", Xm_tr.shape, "| kept cols:", Xm_tr.shape[1])
print("  ESM       :", Xe_tr.shape, "| kept cols:", Xe_tr.shape[1])

# ---------- 2) Prepare scalers (for SVM/MLP only) ----------
sc_k = StandardScaler().fit(Xk_tr) if Xk_tr.shape[1] > 0 else None
sc_p = StandardScaler().fit(Xp_tr) if Xp_tr.shape[1] > 0 else None
sc_m = StandardScaler().fit(Xm_tr) if Xm_tr.shape[1] > 0 else None
sc_e = StandardScaler().fit(Xe_tr) if Xe_tr.shape[1] > 0 else None

def scale_block(sc, Xtr, Xte):
    if sc is None: return Xtr, Xte
    return sc.transform(Xtr), sc.transform(Xte)

# Raw blocks (tree-based / XGB)
blocks_raw_tr = {"kmer": Xk_tr, "physchem": Xp_tr, "modlamp": Xm_tr, "esm": Xe_tr}
blocks_raw_te = {"kmer": Xk_te, "physchem": Xp_te, "modlamp": Xm_te, "esm": Xe_te}

# Scaled blocks (SVM / MLP)
Xk_tr_s, Xk_te_s = scale_block(sc_k, Xk_tr, Xk_te)
Xp_tr_s, Xp_te_s = scale_block(sc_p, Xp_tr, Xp_te)
Xm_tr_s, Xm_te_s = scale_block(sc_m, Xm_tr, Xm_te)
Xe_tr_s, Xe_te_s = scale_block(sc_e, Xe_tr, Xe_te)
blocks_scl_tr = {"kmer": Xk_tr_s, "physchem": Xp_tr_s, "modlamp": Xm_tr_s, "esm": Xe_tr_s}
blocks_scl_te = {"kmer": Xk_te_s, "physchem": Xp_te_s, "modlamp": Xm_te_s, "esm": Xe_te_s}

# ---------- 3) Ensure we have idx_tr / idx_val ----------
if 'idx_tr' not in locals() or 'idx_val' not in locals():
    sss_tmp = StratifiedShuffleSplit(n_splits=1, test_size=VAL_SIZE, random_state=RANDOM_STATE)
    idx_tr, idx_val = next(sss_tmp.split(Xk_tr, y_train))
    y_tr, y_val = y_train[idx_tr], y_train[idx_val]

# ---------- 4) Models ----------
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale_pos_weight = float(neg) / float(max(pos, 1))

def make_models():
    models = []
    if HAVE_XGB:
        models.append(("XGB",
            XGBClassifier(
                n_estimators=300, max_depth=6, learning_rate=0.1,
                subsample=0.8, colsample_bytree=0.8,
                reg_lambda=1.0, random_state=RANDOM_STATE,
                n_jobs=-1, eval_metric="logloss",
                scale_pos_weight=scale_pos_weight
            ),
            "raw"))
    models.append(("RF",
        RandomForestClassifier(
            n_estimators=400, max_depth=None, min_samples_leaf=1,
            class_weight="balanced", n_jobs=-1, random_state=RANDOM_STATE
        ),
        "raw"))
    models.append(("SVM",
        SVC(kernel="rbf", C=1.0, gamma="scale", probability=True,
            class_weight="balanced", random_state=RANDOM_STATE),
        "scaled"))
    models.append(("MLP",
        MLPClassifier(
            hidden_layer_sizes=(256,128), activation="relu",
            learning_rate_init=1e-3, batch_size=128,
            max_iter=100, early_stopping=True, random_state=RANDOM_STATE
        ),
        "scaled"))
    return models

# ---------- 5) All non-empty block combinations ----------
all_blocks = ["kmer", "physchem", "modlamp", "esm"]
all_combos = list(chain.from_iterable(combinations(all_blocks, r) for r in range(1, len(all_blocks)+1)))

# ---------- 6) Run ablation on VALIDATION (FIX: slice *TRAIN* for val) ----------
val_rows = []
for combo in all_combos:
    combo = list(combo)
    combo_name = "+".join(combo)

    # 👉 Use TRAIN blocks for both train/val splits
    Xtr_raw = concat_blocks([blocks_raw_tr[b][idx_tr] for b in combo])
    Xva_raw = concat_blocks([blocks_raw_tr[b][idx_val] for b in combo])

    Xtr_scl = concat_blocks([blocks_scl_tr[b][idx_tr] for b in combo])
    Xva_scl = concat_blocks([blocks_scl_tr[b][idx_val] for b in combo])

    for mdl_name, mdl, kind in make_models():
        Xtr_use, Xva_use = (Xtr_raw, Xva_raw) if kind == "raw" else (Xtr_scl, Xva_scl)
        mdl.fit(Xtr_use, y_tr)
        p_val = mdl.predict_proba(Xva_use)[:, 1]
        m = report_metrics_fast(y_val, p_val, thr=THRESH)
        m.update(dict(model=mdl_name, combo=combo_name, n_feat=Xtr_use.shape[1], set="val"))
        val_rows.append(m)
        print(f"[Ablation/VAL] {combo_name:<30} | {mdl_name:<4} | "
              f"feat={Xtr_use.shape[1]:6d} | MCC={m['MCC']:.4f} AUC={m['AUC']:.4f} ACC={m['ACC']:.4f}")

val_df = pd.DataFrame(val_rows).sort_values(["MCC","AUC","ACC"], ascending=False).reset_index(drop=True)
val_df.to_csv("ablation_validation_summary.csv", index=False)
print("\nSaved ablation_validation_summary.csv (sorted by MCC, then AUC, ACC)")
print(val_df.head(15).to_string(index=False))

# ---------- 7) Optional TEST evaluation ----------
if "label" in df_test.columns:
    y_test = df_test["label"].values.astype(int)
    test_rows = []

    # Refit on FULL TRAIN, evaluate on TEST
    for combo in all_combos:
        combo = list(combo)
        combo_name = "+".join(combo)

        Xtr_raw_full = concat_blocks([blocks_raw_tr[b] for b in combo])
        Xte_raw_full = concat_blocks([blocks_raw_te[b] for b in combo])
        Xtr_scl_full = concat_blocks([blocks_scl_tr[b] for b in combo])
        Xte_scl_full = concat_blocks([blocks_scl_te[b] for b in combo])

        for mdl_name, mdl, kind in make_models():
            Xtr_use, Xte_use = (Xtr_raw_full, Xte_raw_full) if kind == "raw" else (Xtr_scl_full, Xte_scl_full)
            mdl.fit(Xtr_use, y_train)
            p_test = mdl.predict_proba(Xte_use)[:, 1]
            m = report_metrics_fast(y_test, p_test, thr=THRESH)
            m.update(dict(model=mdl_name, combo=combo_name, n_feat=Xtr_use.shape[1], set="test"))
            test_rows.append(m)
            print(f"[Ablation/TEST] {combo_name:<30} | {mdl_name:<4} | "
                  f"feat={Xtr_use.shape[1]:6d} | MCC={m['MCC']:.4f} AUC={m['AUC']:.4f} ACC={m['ACC']:.4f}")

    test_df = pd.DataFrame(test_rows).sort_values(["MCC","AUC","ACC"], ascending=False).reset_index(drop=True)
    test_df.to_csv("ablation_test_summary.csv", index=False)
    print("\nSaved ablation_test_summary.csv (sorted by MCC, then AUC, ACC)")
    print(test_df.head(15).to_string(index=False))
else:
    print("\n[df_test has no labels] Skipping TEST-set ablation metrics.")


[Ablation] After per-block NZV:
  k-mer     : (2872, 35358) | kept cols: 35358
  physchem  : (2872, 11) | kept cols: 11
  modlAMP   : (2872, 10) | kept cols: 10
  ESM       : (2872, 320) | kept cols: 320
[Ablation/VAL] kmer                           | XGB  | feat= 35358 | MCC=0.2025 AUC=0.6617 ACC=0.5868
[Ablation/VAL] kmer                           | RF   | feat= 35358 | MCC=0.2580 AUC=0.7182 ACC=0.6181
[Ablation/VAL] kmer                           | SVM  | feat= 35358 | MCC=0.2231 AUC=0.6676 ACC=0.6042
[Ablation/VAL] kmer                           | MLP  | feat= 35358 | MCC=0.3011 AUC=0.7000 ACC=0.6458
[Ablation/VAL] physchem                       | XGB  | feat=    11 | MCC=0.3053 AUC=0.7130 ACC=0.6458
[Ablation/VAL] physchem                       | RF   | feat=    11 | MCC=0.2690 AUC=0.7036 ACC=0.6319
[Ablation/VAL] physchem                       | SVM  | feat=    11 | MCC=0.2522 AUC=0.6654 ACC=0.6285
[Ablation/VAL] physchem                       | MLP  | feat=    11 | MCC=0.2006 AU